In [ ]:
# External Dependencies: nltk, transformers, np

import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
import csv
import json
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from itertools import repeat

# Reading data from file

In [ ]:
path='C:/UMass/CS685/Project/github_repo/CS685_Sarcasm/'
train_df=pd.read_csv(path+"/train_main_sub.tsv")
test_df=pd.read_csv(path+"/test_main_sub.tsv")

In [ ]:
train_df=train_df.dropna(subset=[2])
test_df=test_df.dropna(subset=[2])

In [ ]:
train_texts=train_df['reddit'].values.tolist()
test_texts=test_df['reddit'].values.tolist()


In [ ]:
seed_words=['angry','happy','fear','disgust','sad','surprise']


# Reading W2V model

In [ ]:
import gensim

# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('./w2v/GoogleNews-vectors-negative300.bin', binary=True)
word_vectors = model.wv

In [ ]:
import string
punct =[]
punct += list(string.punctuation)
punct += '’'
punct.remove("'")
def remove_punctuations(text):
    for punctuation in punct:
        text = text.replace(punctuation, ' ')
    return text

In [ ]:
def text_preprocessing(text):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    """
    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)
    text = re.sub(r"http\S+", "", text)
    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    text=remove_punctuations(text)
    toks = nltk.tokenize.word_tokenize(text)
#     print(toks)
    text = [word for word in toks if word not in stopwords.words('english')]
    
    return text

In [ ]:
max_seq_length=64

In [ ]:
pad_tok='<pad>'

In [ ]:
train_tokens=[]
i=0
for text in train_texts: 
    pp_t=text_preprocessing(text)
    print(i)
    i+=1
    if len(pp_t)<max_seq_length:
        pad_length=max_seq_length - len(pp_t)
#         print(pad_length)
        padded_tokens = pp_t + [pad_tok] * pad_length
        train_tokens.append(padded_tokens)
#         train_tokens.append(pad_tok*pad_length)
    elif len(pp_t)>=max_seq_length:
        train_tokens.append(pp_t[:max_seq_length])

for text in test_texts: 
    pp_t=text_preprocessing(text)
#     print(pp_t)
    if len(pp_t)<max_seq_length:
        pad_length=max_seq_length - len(pp_t)
#         print(pad_length)
        padded_tokens = pp_t + [pad_tok] * pad_length
        train_tokens.append(padded_tokens)
#         train_tokens.append(pad_tok*pad_length)
    elif len(pp_t)>=max_seq_length:
        train_tokens.append(pp_t[:max_seq_length])
    
#     stop

In [ ]:
seeds=[]
for seed in seed_words:
    seeds.append(word_vectors[seed])
seed_word_vectors=np.array(seeds)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine(tokens,seed_word_vectors):
    sents=[]
    for token in tokens:
#         print(token)
        try:
            wvector=word_vectors.get_vector(token)
        except KeyError:
            wvector = np.zeros((300,))
        
        sents.append(wvector)
    sent_vector=np.array(sents)
    return cosine_similarity(sent_vector,seed_word_vectors)
#     x=[]
#     for seed in seed_words:
#         x.append(model.similarity(word, seed_words))
    
#     return np.array(x)



In [ ]:
sent_emo_list=[]
for toks in train_tokens:
#     print(len(toks))
    sim_vecs=get_cosine(toks,seed_word_vectors)
#     print((sim_vecs).shape)
    sent_emo_list.append(np.mean((sim_vecs),axis=0))
sent_emo_vec=np.array(sent_emo_list)

In [ ]:
emo_vecs=np.mean(sent_emo_vec, axis=1).shape

In [ ]:
np.save('emo_vecs_preprocessed', sent_emo_vec)